In [1]:
import os
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from IPython.display import display
from PIL import Image

Using TensorFlow backend.


In [2]:
train_dir = "../../../data_kaggle/train/train"
test_dir = "../../data_kaggle/test/test/test"

In [3]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

In [26]:
batch_size=64

train_generator = train_datagen.flow_from_directory(
        os.getcwd()+os.path.sep+train_dir,
        target_size=(32, 32),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training') 

validation_generator = train_datagen.flow_from_directory(
        os.getcwd()+os.path.sep+train_dir,
        target_size=(32, 32),
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation') 

Found 3803 images belonging to 12 classes.
Found 947 images belonging to 12 classes.


In [27]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(20, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(10, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(12, activation='relu'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_39 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 13, 13, 20)        5780      
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 6, 6, 20)          0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 4, 4, 10)          1810      
_________________________________________________________________
flatten_10 (Flatten)         (None, 160)               0         
_________________________________________________________________
dense_30 (Dense)             (None, 64)                10304     
__________

In [28]:
n_epochs=5

model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = n_epochs)

Epoch 1/5
59/59 [==============================] - 44s 753ms/step - loss: 7.4909 - acc: 0.1245 - val_loss: 7.3288 - val_acc: 0.1317
Epoch 2/5
59/59 [==============================] - 39s 669ms/step - loss: 7.2814 - acc: 0.1273 - val_loss: 7.2767 - val_acc: 0.1438
Epoch 3/5
59/59 [==============================] - 40s 675ms/step - loss: 7.3205 - acc: 0.1643 - val_loss: 7.2042 - val_acc: 0.2039
Epoch 4/5
59/59 [==============================] - 41s 696ms/step - loss: 7.3258 - acc: 0.2223 - val_loss: 7.3186 - val_acc: 0.2571
Epoch 5/5
59/59 [==============================] - 40s 685ms/step - loss: 6.5814 - acc: 0.2531 - val_loss: 6.2052 - val_acc: 0.2741
